In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re 
import math
from statsmodels.formula.api import ols 
from statsmodels.formula.api import logit 

In [42]:
comments = pd.read_csv('comments_dollar.csv')
assert comments.shape[0] == comments.drop_duplicates().shape[0]

In [3]:
comments.shape

(169919, 12)

# 코멘트 갯수에 따른 펀딩성공의 연관성  

In [5]:
general = pd.read_csv('general_final.csv')

In [6]:
# 성공 조건 만들기 funds_raised_dollar > funds_target_dollar 
general['gensuccess'] = general.funds_raised_dollar > general.funds_target_dollar
general['genfail'] = general.funds_raised_dollar < general.funds_target_dollar
general['dum_gensuccess'] = general['gensuccess'].astype(int)
general['dum_genfail'] = general['genfail'].astype(int)

In [7]:
general.head()

,chid,url,scrape_date,title,created_date,funds_raised,funds_target,picture,org_picture,counts_status_code,...,char_age,success,dum_success,group_category,total_comments,update_counts,gensuccess,genfail,dum_gensuccess,dum_genfail
0,1,https://www.gofundme.com/f/-chris-beaty-memori...,2020-11-03,Chris Beaty Memorial Fund,2020-06-03,175083.0,200000.0,True,False,200,...,153,False,0,Funerals & Memorials,74.0,1.0,False,True,0,1
1,2,https://www.gofundme.com/f/1-prosent-chance,2020-11-03,1 percent chance,2020-06-21,299042.0,2000000.0,True,False,200,...,135,False,0,"Medical, Illness & Healing",0.0,0.0,False,True,0,1
2,3,https://www.gofundme.com/f/100-computers-in-10...,2020-11-04,MELS needs #100ComputersIn100Days,2020-06-19,13845.0,30000.0,True,True,200,...,138,False,0,Others,7.0,4.0,False,True,0,1
3,4,https://www.gofundme.com/f/100-to-victims-of-t...,2020-11-03,100% to Victims of Taal Volcano in Philippines,2020-01-16,25.0,5000.0,True,True,200,...,292,False,0,Accidents & Emergencies,0.0,0.0,False,True,0,1
4,5,https://www.gofundme.com/f/100k-for-center-for...,2020-11-04,$100k for Center for Family Life,2020-11-02,13950.0,100000.0,True,False,200,...,2,False,0,Others,1.0,0.0,False,True,0,1


In [8]:
general.chid = general.chid.astype(int)

In [62]:
# chid id 별 comment 갯수 구하기 
comments_chid =  comments.groupby('chid').count().comment.to_frame().reset_index()
comments_chid

,chid,comment
0,1,74
1,3,7
2,5,1
3,6,14
4,7,32
...,...,...
3676,3901,16
3677,3902,27
3678,3903,41
3679,3904,1


In [21]:
# comments 테이블과 comments_chid 테이블(캠페인 당 코멘트 갯수)를 chid로 merge하기 
comments = pd.merge(comments, comments_chid, on='chid', how='inner')

In [30]:
# 코멘트 파일과 제네럴을 chid 로 머지해서 제네럴의 chid와 success를 가져온다 
comments_success = comments.merge(general[['chid','dum_gensuccess']], on= 'chid',how='inner')
# 최종적으로 chid, comment_count, sucess 가 있는 데이터 가져오기 
comments_success= comments_success[['chid', 'comment_y','dum_gensuccess']]
comments_success = comments_success.groupby('chid')[['comment_y','dum_gensuccess']].mean().reset_index()
comments_success.head()

,chid,comment_y,dum_gensuccess
0,1,74,0
1,3,7,0
2,5,1,0
3,6,14,0
4,7,32,1


In [31]:
# comment_y에서 comment_count 컬럼명 변경
comments_success.rename(columns = {'comment_y':'comment_count'}, inplace = True)

In [32]:
# result_success
comments_success

,chid,comment_count,dum_gensuccess
0,1,74,0
1,3,7,0
2,5,1,0
3,6,14,0
4,7,32,1
...,...,...,...
3676,3901,16,1
3677,3902,27,1
3678,3903,41,1
3679,3904,1,0


In [36]:
comments_success = comments_success.groupby('comment_count')['dum_gensuccess'].sum().reset_index().rename(columns={'dum_gensuccess':'success_cases'})

In [33]:
# result_fail
# 업데이트 파일과 제네럴을 chid 로 머지해서 제네럴의 chid와 sucess를 가져온다 
comments_fail = comments.merge(general[['chid','dum_genfail']], on= 'chid',how='inner') 
# 최종적으로 chid, updateno, sucess 가 있는 데이터 가져오기 
comments_fail= comments_fail[['chid', 'comment_y','dum_genfail']]
comments_fail = comments_fail.groupby('chid')[['comment_y', 'dum_genfail']].mean().reset_index()
comments_fail.head()

,chid,comment_y,dum_genfail
0,1,74,1
1,3,7,1
2,5,1,1
3,6,14,1
4,7,32,0


In [34]:
# comment_y에서 comment_count 컬럼명 변경
comments_fail.rename(columns = {'comment_y':'comment_count'}, inplace = True)

In [37]:
comments_fail = comments_fail.groupby('comment_count')['dum_genfail'].sum().reset_index().rename(columns={'dum_genfail':'fail_cases'})

In [38]:
# success, fail merge
result_suc_fail = comments_success.merge(comments_fail[['comment_count','fail_cases']], on= 'comment_count',how='inner') 

In [39]:
result_suc_fail

,comment_count,success_cases,fail_cases
0,1,29,68
1,2,32,57
2,3,31,63
3,4,31,49
4,5,34,57
...,...,...,...
246,316,1,7
247,317,3,3
248,318,6,5
249,319,6,7


In [40]:
# 비율구하기(success_cases/(success_cases+fail_cases)) 최종 차트
result_suc_fail['sucess_percentage'] = (result_suc_fail.success_cases/(result_suc_fail.success_cases + result_suc_fail.fail_cases) *100)
result_suc_fail = result_suc_fail[['comment_count', 'success_cases', 'sucess_percentage']]
result_suc_fail

,comment_count,success_cases,sucess_percentage
0,1,29,29.896907
1,2,32,35.955056
2,3,31,32.978723
3,4,31,38.750000
4,5,34,37.362637
...,...,...,...
246,316,1,12.500000
247,317,3,50.000000
248,318,6,54.545455
249,319,6,46.153846


In [52]:
# 아웃라이어 제거
# stats = result_suc_fail['sucess_percentage'].describe()
# iqr = stats['75%'] - stats['25%']
# outlier_high = stats['75%'] + 3*iqr
# outlier_low = stats['25%'] - 3*iqr
# result_suc_fail = result_suc_fail[ (result_suc_fail['sucess_percentage'] < outlier_high) & (result_suc_fail['sucess_percentage'] > outlier_low) ]

In [54]:
# result_suc_fail

# 코멘트 길이 당 성공 캠페인

In [43]:
# 코멘트의 길이 컬럼 추가
comments["comment_length"]= comments["comment"].str.len() 

In [44]:
# comments랑 general merge
comments_general = comments.merge(general, on= 'chid',how='inner') 
comments_general = comments_general[['chid', 'comment_id', 'comment_length', 'gensuccess', 'dum_gensuccess', 'genfail', 'dum_genfail']]
comments_general

,chid,comment_id,comment_length,gensuccess,dum_gensuccess,genfail,dum_genfail
0,1,0,146.0,False,0,True,1
1,1,1,18.0,False,0,True,1
2,1,2,304.0,False,0,True,1
3,1,3,38.0,False,0,True,1
4,1,4,30.0,False,0,True,1
...,...,...,...,...,...,...,...
169914,3905,169914,54.0,False,0,True,1
169915,3905,169915,88.0,False,0,True,1
169916,3905,169916,69.0,False,0,True,1
169917,3905,169917,111.0,False,0,True,1


In [149]:
# 코멘트 길이 평균
comments_general.comment_length.mean()

100.70869642857143

In [45]:
# 코멘트 갯수 당 펀딩에 실패한 캠페인 갯수와 비율 
# fail_cases 컬럼 추가
length_fail = comments_general.groupby('comment_length')['dum_genfail'].sum().reset_index().rename(columns={'dum_genfail':'fail_cases'})
length_fail

,comment_length,fail_cases
0,1.0,190
1,2.0,495
2,3.0,298
3,4.0,207
4,5.0,122
...,...,...
988,1756.0,1
989,1795.0,0
990,2186.0,1
991,2562.0,0


In [46]:
length_success = comments_general.groupby('comment_length')['dum_gensuccess'].sum().reset_index().rename(columns={'dum_gensuccess':'success_cases'})
length_success

,comment_length,success_cases
0,1.0,150
1,2.0,498
2,3.0,175
3,4.0,157
4,5.0,89
...,...,...
988,1756.0,0
989,1795.0,1
990,2186.0,0
991,2562.0,1


In [47]:
length_suc_fail = length_success.merge(length_fail[['comment_length','fail_cases']], on= 'comment_length',how='inner') 
length_suc_fail

,comment_length,success_cases,fail_cases
0,1.0,150,190
1,2.0,498,495
2,3.0,175,298
3,4.0,157,207
4,5.0,89,122
...,...,...,...
988,1756.0,0,1
989,1795.0,1,0
990,2186.0,0,1
991,2562.0,1,0


In [48]:
# fail, success merge, 비율구하기
# 코멘트 길이 최종 차트
length_suc_fail['sucess_percentage'] = (length_suc_fail.success_cases/(length_suc_fail.success_cases + length_suc_fail.fail_cases) *100)
length_suc_fail = length_suc_fail[['comment_length', 'success_cases', 'sucess_percentage']]
length_suc_fail

,comment_length,success_cases,sucess_percentage
0,1.0,150,44.117647
1,2.0,498,50.151057
2,3.0,175,36.997886
3,4.0,157,43.131868
4,5.0,89,42.180095
...,...,...,...
988,1756.0,0,0.000000
989,1795.0,1,100.000000
990,2186.0,0,0.000000
991,2562.0,1,100.000000


In [50]:
# 아웃라이어 제거
stats = length_suc_fail['sucess_percentage'].describe()
iqr = stats['75%'] - stats['25%']
outlier_high = stats['75%'] + 3*iqr
outlier_low = stats['25%'] - 3*iqr
length_suc_fail = length_suc_fail[ (length_suc_fail['sucess_percentage'] < outlier_high) & (length_suc_fail['sucess_percentage'] > outlier_low) ]

In [51]:
length_suc_fail

,comment_length,success_cases,sucess_percentage
0,1.0,150,44.117647
1,2.0,498,50.151057
2,3.0,175,36.997886
3,4.0,157,43.131868
4,5.0,89,42.180095
...,...,...,...
982,1562.0,0,0.000000
983,1579.0,0,0.000000
984,1584.0,0,0.000000
988,1756.0,0,0.000000


In [ ]:
# success_percentage 0%, 100% 제거

# 카테고리별 코멘트 평균 갯수

In [65]:
general.head()

,chid,url,scrape_date,title,created_date,funds_raised,funds_target,picture,org_picture,counts_status_code,...,char_age,success,dum_success,group_category,total_comments,update_counts,gensuccess,genfail,dum_gensuccess,dum_genfail
0,1,https://www.gofundme.com/f/-chris-beaty-memori...,2020-11-03,Chris Beaty Memorial Fund,2020-06-03,175083.0,200000.0,True,False,200,...,153,False,0,Funerals & Memorials,74.0,1.0,False,True,0,1
1,2,https://www.gofundme.com/f/1-prosent-chance,2020-11-03,1 percent chance,2020-06-21,299042.0,2000000.0,True,False,200,...,135,False,0,"Medical, Illness & Healing",0.0,0.0,False,True,0,1
2,3,https://www.gofundme.com/f/100-computers-in-10...,2020-11-04,MELS needs #100ComputersIn100Days,2020-06-19,13845.0,30000.0,True,True,200,...,138,False,0,Others,7.0,4.0,False,True,0,1
3,4,https://www.gofundme.com/f/100-to-victims-of-t...,2020-11-03,100% to Victims of Taal Volcano in Philippines,2020-01-16,25.0,5000.0,True,True,200,...,292,False,0,Accidents & Emergencies,0.0,0.0,False,True,0,1
4,5,https://www.gofundme.com/f/100k-for-center-for...,2020-11-04,$100k for Center for Family Life,2020-11-02,13950.0,100000.0,True,False,200,...,2,False,0,Others,1.0,0.0,False,True,0,1


In [67]:
comments_chid.head()

,chid,comment
0,1,74
1,3,7
2,5,1
3,6,14
4,7,32


In [69]:
# general과 comments_chid join 후 category 컬럼 남기기
general_comments = comments_chid.merge(general[['chid','category']], on= 'chid', how='inner')
general_comments

,chid,comment,category
0,1,74,Funerals & Memorials
1,3,7,Education & Learning
2,5,1,Community & Neighbors
3,6,14,"Medical, Illness & Healing"
4,7,32,Community & Neighbors
...,...,...,...
3676,3901,16,"Medical, Illness & Healing"
3677,3902,27,Funerals & Memorials
3678,3903,41,"Medical, Illness & Healing"
3679,3904,1,Funerals & Memorials


In [116]:
# Funerals & Meorials의 chid당 코멘트의 평균
Funerals_Memorials = general_comments.loc[general_comments['category'].str.contains('Funerals & Memorials', na=False)]
Funerals_Memorials

,chid,comment,category
0,1,74,Funerals & Memorials
9,12,18,Funerals & Memorials
12,15,48,Funerals & Memorials
15,18,202,Funerals & Memorials
17,20,21,Funerals & Memorials
...,...,...,...
3666,3891,20,Funerals & Memorials
3669,3894,35,Funerals & Memorials
3674,3899,24,Funerals & Memorials
3677,3902,27,Funerals & Memorials


In [85]:
Funerals_Memorials.comment.mean()

41.652791878172586

In [144]:
# Medical, Illness & Healing의 chid당 코멘트의 평균
Medical_Illness_Healing = general_comments.loc[general_comments['category'].str.contains('Medical, Illness & Healing', na=False)]
Medical_Illness_Healing

,chid,comment,category
3,6,14,"Medical, Illness & Healing"
6,9,95,"Medical, Illness & Healing"
7,10,72,"Medical, Illness & Healing"
8,11,140,"Medical, Illness & Healing"
13,16,153,"Medical, Illness & Healing"
...,...,...,...
3673,3898,1,"Medical, Illness & Healing"
3675,3900,5,"Medical, Illness & Healing"
3676,3901,16,"Medical, Illness & Healing"
3678,3903,41,"Medical, Illness & Healing"


In [87]:
Medical_Illness_Healing.comment.mean()

60.373708920187795

In [104]:
# Accidents & Emergencies의 chid당 코멘트의 평균
Accidents_Emergencies = general_comments.loc[general_comments['category'].str.contains('Accidents & Emergencies', na=False)]
Accidents_Emergencies

,chid,comment,category
5,8,58,Accidents & Emergencies
10,13,319,Accidents & Emergencies
11,14,18,Accidents & Emergencies
14,17,21,Accidents & Emergencies
19,24,80,Accidents & Emergencies
...,...,...,...
3660,3885,152,Accidents & Emergencies
3661,3886,85,Accidents & Emergencies
3662,3887,20,Accidents & Emergencies
3663,3888,37,Accidents & Emergencies


In [105]:
Accidents_Emergencies = Accidents_Emergencies[['comment']]
Accidents_Emergencies.rename(columns = {'comment':'Accidents & Emergencies'}, inplace = True)
Accidents_Emergencies 

/Users/jiyoungyeo/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Accidents & Emergencies
5,58
10,319
11,18
14,21
19,80
...,...
3660,152
3661,85
3662,20
3663,37


In [132]:
Accidents_Emergencies = Accidents_Emergencies.reset_index()
Accidents_Emergencies

,index,level_0,Accidents & Emergencies
0,0,0,58
1,1,1,319
2,2,2,18
3,3,3,21
4,4,4,80
...,...,...,...
1041,1041,1041,152
1042,1042,1042,85
1043,1043,1043,20
1044,1044,1044,37


In [117]:
Funerals_Memorials = Funerals_Memorials[['comment']]
Funerals_Memorials 

,comment
0,74
9,18
12,48
15,202
17,21
...,...
3666,20
3669,35
3674,24
3677,27


In [131]:
Funerals_Memorials.rename(columns = {'comment':'Funerals & Memorials'}, inplace = True)
Funerals_Memorials = Funerals_Memorials.reset_index()
Funerals_Memorials

,index,Funerals & Memorials
0,0,74
1,1,18
2,2,48
3,3,202
4,4,21
...,...,...
980,980,20
981,981,35
982,982,24
983,983,27


In [145]:
Medical_Illness_Healing = Medical_Illness_Healing[['comment']]
Medical_Illness_Healing.rename(columns = {'comment':'Medical & Illness & Healing'}, inplace = True)
Medical_Illness_Healing = Medical_Illness_Healing.reset_index()
Medical_Illness_Healing

/Users/jiyoungyeo/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,index,Medical & Illness & Healing
0,3,14
1,6,95
2,7,72
3,8,140
4,13,153
...,...,...
1060,3673,1
1061,3675,5
1062,3676,16
1063,3678,41


In [133]:
outer_join = pd.merge(Funerals_Memorials, Accidents_Emergencies, left_on='index', right_on='index', how='outer')
print(outer_join)

      index  Funerals & Memorials  level_0  Accidents & Emergencies
0         0                  74.0        0                       58
1         1                  18.0        1                      319
2         2                  48.0        2                       18
3         3                 202.0        3                       21
4         4                  21.0        4                       80
...     ...                   ...      ...                      ...
1041   1041                   NaN     1041                      152
1042   1042                   NaN     1042                       85
1043   1043                   NaN     1043                       20
1044   1044                   NaN     1044                       37
1045   1045                   NaN     1045                       20

[1046 rows x 4 columns]


In [147]:
final = pd.merge(outer_join, Medical_Illness_Healing, left_on='index', right_on='index', how='outer')
final = final[['Funerals & Memorials', 'Accidents & Emergencies', 'Medical & Illness & Healing']]
final

,Funerals & Memorials,Accidents & Emergencies,Medical & Illness & Healing
0,74.0,58.0,NaN
1,18.0,319.0,NaN
2,48.0,18.0,NaN
3,202.0,21.0,14.0
4,21.0,80.0,NaN
...,...,...,...
1777,NaN,NaN,1.0
1778,NaN,NaN,5.0
1779,NaN,NaN,16.0
1780,NaN,NaN,41.0


In [148]:
final.to_excel("카테고리별 코멘트.xlsx") 

In [92]:
Accidents_Emergencies.comment.mean()

48.33938814531549

In [99]:
df = pd.DataFrame({'Funerals & Memorials': [42],
                   'Accidents & Emergencies': [48],
                   'Medical & Illness & Healin': [60]},
                  index=['comment_count'])
df

,Funerals & Memorials,Accidents & Emergencies,Medical & Illness & Healin
comment_count,42,48,60


In [ ]:
sns.barplot(x='update_count', y='funds_raised_dollar', data=line)

In [74]:
# groupby category, comment sum
# category_comment = comments_success.groupby('comment_count')['comment'].sum().reset_index().rename(columns={'dum_gensuccess':'success_cases'})
# chid id 별 comment 갯수 구하기 
category_comment =  general_comments.groupby('category').sum().comment.to_frame().reset_index()
category_comment

,category,comment
0,Accidents & Emergencies,50563
1,Animals & Pets,2750
2,"Babies, Kids & Family",442
3,Business & Entrepreneurs,108
4,Celebrations & Events,66
5,Community & Neighbors,2450
6,Competitions & Pageants,70
7,"Creative Arts, Music & Film",915
8,"Dreams, Hopes & Wishes",143
9,Education & Learning,1133


# 카테고리별 social_share

In [56]:
general

0             Funerals & Memorials
1       Medical, Illness & Healing
2             Education & Learning
3          Accidents & Emergencies
4            Community & Neighbors
                   ...            
3899    Medical, Illness & Healing
3900          Funerals & Memorials
3901    Medical, Illness & Healing
3902          Funerals & Memorials
3903    Medical, Illness & Healing
Name: category, Length: 3904, dtype: object